# 制約付きMDPでの探索と活用

参考

* [Exploration-Exploitation in Constrained MDPs](https://arxiv.org/abs/2003.02189)

強化学習では累積報酬和を最大化するような方策を学習することが目標ですが、実際のアプリケーションでは方策の挙動に対して何らかの制約が求められる場合があります。
制約付きMDP（CMDP）では制約を満たしながらの累積報酬和の最大化が求められるわけですが、この状況での探索はどうすればよいでしょうか？


表記

* 有限MDP: $\mathcal{M}=\left(\mathcal{S}, \mathcal{A}, c, p, s_1, H\right)$
    1. 有限状態集合: $S=\{1, \dots, |S|\}$
    2. 有限行動集合: $A=\{1, \dots, |A|\}$
    3. 非定常なコスト関数（元論文では確率変数ですが、ややこしいので決定的にします）: $c_h(s, a)$
    4. 非定常遷移確率: $p_h(s'|s, a)$
    5. 初期状態: $s_1$
    6. ホライゾン: $H$
    7. すべての状態行動の中で最大の次状態への遷移の数: $\mathcal{N}:=\max _{s, a, h}\left|\left\{s^{\prime}: p_h\left(s^{\prime} \mid s, a\right)>0\right\}\right|$
* 占有率：$q_h^\pi(s, a ; p):=\mathbb{E}\left[\mathbb{1}\left\{s_h=s, a_h=a\right\} \mid s_1=s_1, p, \pi\right]=\operatorname{Pr}\left\{s_h=s, a_h=a \mid s_1=s_1, p, \pi\right\}$
* 価値関数：$V_1^\pi\left(s_1 ; p, c\right)=\sum_{h, s, a} q_h^\pi(s, a ; p) c_h(s, a):=c^T q^\pi(p)$

これは次の式で導出可能
$$
\begin{aligned}
& \mathbb{E}\left[\sum_{h=1}^H c_h\left(s_h, a_h\right) \mid s_1=s_1, \pi, p\right]=\sum_{h=1}^H \mathbb{E}\left[c_h\left(s_h, a_h\right) \mid s_1=s_1, \pi, p\right] \\
& =\sum_{h=1}^H \sum_{s, a} c_h(s, a) \operatorname{Pr}\left\{s_h=s, a_h=a \mid s_1=s_1, p, \pi\right\} \\
& \sum_{h=1}^H \sum_{s, a} c_h(s, a) q_h^\pi(s, a ; p)=c^T q^\pi(p),
\end{aligned}
$$


## 有限ホライゾンConstrained MDP

* $\{d_i, \alpha_i\}_{i=1}^I$：$I$個の制約
    * $d_i \in \mathbb{R}^{SAH}$
    * $\alpha_i \in [0, H]$
    * $i$番目の制約（元論文では確率変数ですが、ややこしいので決定的にします）$d_{i, h}(s, a)$
    * $V_h^\pi\left(s ; p, d_i\right):=\mathbb{E}\left[\sum_{h^{\prime}=h}^H d_{i, h^{\prime}}\left(s_{h^{\prime}}, a_{h^{\prime}}\right) \mid s_h=s, p, \pi\right]$.

CMDPの目的は次の最適方策の導出です。

$$
\begin{gathered}
\pi^{\star} \in \underset{\pi \in \Pi^{\mathrm{MR}}}{\arg \min } c^T q^\pi(p) \\
\text { s.t. } D q^\pi(p) \leq \alpha
\end{gathered}
$$

ここで、
$$
D=\left[\begin{array}{c}
d_1^T \\
\vdots \\
d_I^T
\end{array}\right], \quad \alpha=\left[\begin{array}{c}
\alpha_1 \\
\vdots \\
\alpha_I
\end{array}\right]
$$

以下では最適方策が方策集合の中にあることを仮定します（Feasibility）。

また、最適方策は決定的でない可能性があることに注意しましょう。制約のせいで、通常のMDPで成り立つ性質がCMDPでも成り立つかは自明ではありません。

CMDPの性能を測るために、次のリグレットを考えます。

$$
\begin{aligned}
& \operatorname{Reg}_{+}(K ; c)=\sum_{k=1}^K\left[V_1^{\pi_k}\left(s_1 ; p, c\right)-V_1^{\star}\left(s_1\right)\right]_{+} \\
& \operatorname{Reg}_{+}(K ; d)=\max _{i \in[I]} \sum_{k=1}^K\left[V_1^{\pi_k}\left(s_1 ; p, d_i\right)-\alpha_i\right]_{+},
\end{aligned}
$$

こっちのリグレットを考えると大変なので、次のリグレットを考えることもあります。

$$
\begin{aligned}
& \operatorname{Reg}(K ; c)=\sum_{k=1} V_1^{\pi_k}\left(s_1 ; p, c\right)-V_1^{\star}\left(s_1\right) \\
& \operatorname{Reg}(K ; d)=\max _{i \in[I]}\left[\sum_{k=1}^K V_1^{\pi_k}\left(s_1 ; p, d_i\right)-\alpha_i\right]
\end{aligned}
$$

これは上で定義した$\operatorname{Reg}_+$のほうが強い制約であることに注意しましょう。下のリグレットでは、負と正のviolation同士が打ち消し合うことがあります。

## CMDPでの線型計画法

まず、任意の方策について占有率は次の方程式を満たします。

$$
\begin{aligned}
\sum_a q_h^\pi(s, a) & =\sum_{s^{\prime}, a^{\prime}} p_{h-1}\left(s \mid s^{\prime}, a^{\prime}\right) q_{h-1}^\pi\left(s^{\prime}, a^{\prime}\right) & & \forall s \in \mathcal{S} \\
q_h^\pi(s, a) & \geq 0 & & \forall s, a
\end{aligned}
$$

この方程式を満たす$q^\pi$は確率測度であることに注意しましょう。
確率測度全体の集合を$\Delta^\mu(\mathcal{M})$とします。（これは凸集合になります。６ページの参考文献参照。）

線型計画法の解で得られた占有率を使って方策を表すことができます。

$$
\pi_h^q(a \mid s)=\frac{q_h(s, a)}{\sum_b q_h(s, b)}, \quad \forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]
$$

これを使うと、CMDPの問題は次の問題と等価になります。

$$
\begin{array}{lll}
& \min _q \sum_{s, a, h} q_h(s, a) c_h(s, a) & \\
\text { s.t. } & \sum_{s, a, h} q_h(s, a) d_{i, h}(s, a) \leq \alpha_i & \forall i \in[I]\\
& \sum_a q_h(s, a)=\sum_{s^{\prime}, a^{\prime}} p_{h-1}\left(s \mid s^{\prime}, a^{\prime}\right) q_{h-1}\left(s^{\prime}, a^{\prime}\right) & \forall h \in[H] \backslash\{1\}\\
& \sum_a q_1(s, a)=\mu(s) & \forall s \in \mathcal{S} \\
& q_h(s, a) \geq 0 & \forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]
\end{array}
$$

**その他の表記**

任意の確率変数$X$について、

$$\mathbb{E}\left[X\left(s_h^k, a_h^k\right) \mid \mathcal{F}_{k-1}\right]=\sum_{s, a} q_h^{\pi_k}(s, a ; p) X(s, a)$$

が成り立つことに注意しましょう。

* $\widetilde{O}(X)$は$X$に対数多項式的に依存する量とします
* $\lesssim$は対数多項式を無視した$\leq$とします。


## CMDPでのUCRLアルゴリズム（OPTCMDP）

まずはCMDP版のUCRL2アルゴリズムであるOptCMDPを見てみましょう。

* $k$エピソード以前に訪れた状態行動の数：$n_h^{k-1}(s, a)=\sum_{k^{\prime}=1}^{k-1} \mathbb{1}\left(s_h^{k^{\prime}}=s, a_h^{k^{\prime}}=a\right)$
* これを使った経験平均値

$$
\begin{aligned}
\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right) & =\frac{\sum_{k^{\prime}=1}^{k-1} \mathbb{1}\left(s_h^{k^{\prime}}=s, a_h^{k^{\prime}}=a, s_{h+1}^{k^{\prime}}=s^{\prime}\right)}{n_h^{k-1}(s, a) \vee 1} \\
\end{aligned}
$$

まず、OptCMDPはUCRLみたいに信頼集合を構築します。

$$
\begin{aligned}
B_{h, k}^p(s, a) & =\left\{\widetilde{p}(\cdot \mid s, a) \in \Delta_S: \forall s^{\prime} \in \mathcal{S},\left|\widetilde{p}(\cdot \mid s, a)-\bar{p}_h^{k-1}(\cdot \mid s, a)\right| \leq \beta_{h, k}^p\left(s, a, s^{\prime}\right)\right\}
\end{aligned}
$$

ここで、その範囲$\beta$は

$$\beta_{h, k}^p\left(s, a, s^{\prime}\right):=2 \sqrt{\frac{\operatorname{Var}\left(\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right) L_\delta^p}{n_h^{k-1}(s, a) \vee 1}}+\frac{14 / 3 L_\delta^p}{n_h^{k-1}(s, a) \vee 1}$$

によって構築します。
ここで、$\operatorname{Var}\left(\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right)=\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right) \cdot\left(1-\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right)$です。（Bernsteinを使ったUCRLでも似たような項が出てきます。）
また、$L_\delta^p=\ln \left(\frac{6 S A H K}{\delta}\right)$です。

この信頼集合を使って、
$$
\mathcal{M}_k=\left\{M=(\mathcal{S}, \mathcal{A}, c, d, \widetilde{p}): \widetilde{p}_h(\cdot \mid s, a) \in B_{h, k}^p(s, a)\right\}
$$

を定義します。
信頼集合が定義されたら、次の最適化問題を解くことで方策を更新します。

$$
\begin{aligned}
\min _{\widetilde{p} \in B_k^p, \pi \in \Pi^{\mathrm{MR}}} & \sum_{h, s, a} {c}_h(s, a) q_h^\pi(s, a ; \widetilde{p}) \\
\text { s.t. } & \sum_{h, s, a} {d}_{i, h}(s, a) q_h^\pi(s, a ; \widetilde{p}) \leq \alpha_i, \quad \forall i \in[H]
\end{aligned}
$$


この問題で得られる方策を$\pi_k$とし、得られるMDPを

$$
\widetilde{M}_k=\left(\mathcal{S}, \mathcal{A}, {c}_k, {d}_k, \widetilde{p}_k\right)
$$

とします。このとき、方策は

$$
V_1^{\pi_k}\left(s_1 ; {c}_k, {p}_k\right):={c}_k^{\top} q^{\pi_k}\left(\widetilde{p}_k\right) \leq c^{\top} q^{\pi^{\star}}(p):=V_1^{\star}\left(s_1 ; c, p\right)
$$

の意味で楽観的になっています。

これを証明しましょう。



### ステップ１：Good eventの定義

まず、Failure eventを定義します。

$$
\begin{aligned}
& F_k^p=\left\{\exists s, a, s^{\prime}, h:\left|p_h\left(s^{\prime} \mid s, a\right)-\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right| \geq \beta_{h, k}^p\left(s, a, s^{\prime}\right)\right\} \\
& F_k^N=\left\{\exists s, a, h: n_h^{k-1}(s, a) \leq \frac{1}{2} \sum_{j<k} q_h^{\pi_k}(s, a \mid p)-H \ln \frac{S A H}{\delta^{\prime}}\right\}
\end{aligned}
$$

また、$F^P=\bigcup_{k=1}^K F_k^p$とします。
つまり、

* $F^p_k$：$k$エピソード目で、遷移確率が推定値から大幅にずれているイベント。証明は[Learning Adversarial MDPs with Bandit Feedback and Unknown Transition](https://arxiv.org/abs/1912.01192)参照。
* $F^p$：$K$エピソード目までに一度でも推定値が大幅にずれているイベント。
* $F^N_k$：全然訪れていない$(s, a)$があるイベント。証明は[Unifying pac and regret: Uniform pac bounds for episodic reinforcement learning](https://arxiv.org/abs/1703.07710)参照。

このとき、
$$
\begin{aligned}
&\operatorname{Pr}\left\{\left|p_h\left(s^{\prime} \mid s, a\right)-\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right| \geq \epsilon_1\right\} \leq \delta^{\prime \prime}\\
&\epsilon_1=\sqrt{\frac{2 \operatorname{Var}\left(\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right) \ln \left(\frac{2}{\delta^{\prime \prime}}\right)}{n_h^{k-1}(s, a) \vee 1}}+\frac{7 \ln \left(\frac{2}{\delta^{\prime \prime}}\right)}{3\left(n_h^{k-1}(s, a)-1\right) \vee 1} .
\end{aligned}
$$
が成立します。また、
$$
\begin{aligned}
&\operatorname{Pr}\left\{\left|p_h\left(s^{\prime} \mid s, a\right)-\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right| \geq \epsilon_2\right\} \leq \delta^{\prime \prime}\\
&\epsilon_2=\sqrt{\frac{2 \operatorname{Var}\left(\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)\right) \ln \left(\frac{2}{\delta^{\prime \prime}}\right)}{n_h^{k-1}(s, a) \vee 1}}+\frac{7 \ln \left(\frac{2}{\delta^{\prime \prime}}\right)}{3\left(n_h^{k-1}(s, a)-1 \vee 1\right)}
\end{aligned}
$$
も成り立ちます。$\epsilon_1 \leq \epsilon_2$なので。

$\delta^{\prime \prime}=\frac{\delta^{\prime}}{(S A H K)^2}$としてUnion Boundをとれば、$\operatorname{Pr}\left\{F^P\right\} \leq \delta^{\prime}$が成り立ちます。


以上のFailure Eventに対して、

$\bar{G}=F^p \bigcup F^N$としてGood eventを定義します。
このGood Eventが成り立っているとき、

$$
\left|\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)-p_h\left(s^{\prime} \mid s, a\right)\right|=C_1 \sqrt{\frac{p_h\left(s^{\prime} \mid s, a\right) L_{\delta, p}}{n_h^k(s, a) \vee 1}}+\frac{C_2 L_{\delta, p}}{n_h^k(s, a) \vee 1}
$$

となる定数$C_1, C_2 > 0$が存在します。ここで$L_{\delta, p}=\ln \left(\frac{6 S A H K}{\delta}\right)$です。


### ステップ２：最適方策が$k$エピソード目で実行可能であることの証明

まず、
$$
\begin{gathered}
\pi^{\star} \in \underset{\pi \in \Pi^{\mathrm{MR}}}{\arg \min } c^T q^\pi(p) \\
\text { s.t. } D q^\pi(p) \leq \alpha
\end{gathered}
$$
と
$$
\begin{aligned}
\min _{\widetilde{p} \in B_k^p, \pi \in \Pi^{\mathrm{MR}}} & \sum_{h, s, a} {c}_h(s, a) q_h^\pi(s, a ; \widetilde{p}) \\
\text { s.t. } & \sum_{h, s, a} {d}_{i, h}(s, a) q_h^\pi(s, a ; \widetilde{p}) \leq \alpha_i, \quad \forall i \in[H]
\end{aligned}
$$
であることを思い出しましょう。また、Good eventが成立しているとき、真のモデル$p$は$B^p_k$の中に入ってることに注意しましょう。
すると、明らかに$\pi^*$は任意の$k$について実行可能です。

（元論文の証明はちょっとよくわかんなかったです。$B$の中のすべての$p'$について制約を満たすようにすると必ずしも実行可能にならなさそうです。）


### ステップ３：楽観的な方策の証明

* $\Pi_D=\left\{\pi: D q^\pi(p) \leq \alpha\right\}$を制約を満たす方策の集合
* $\Pi_D^k=\left\{\pi: {D} q^\pi\left(p^{\prime}\right) \leq \alpha,\; \exists p^{\prime} \in B_k^p\right\}$ を何らかの$p'$に対して制約を満たすような方策の集合（$\pi^\star \in \Pi_D^k$であることに注意しましょう）

Good eventのとき、

$$
V_1^{\pi_k}\left(s_1 ; {c}, \widetilde{p}_k\right) \leq V_1^{\star}\left(s_1\right)
$$

および

$$
V_1^{\pi_k}\left(s_1 ; {c}, \widetilde{p}_k\right) \leq V_1^{\pi_k}\left(s_1 ; c, p\right)
$$

であることを示しましょう。

$$
\begin{aligned}
V^{\pi_k}\left(s_1; c, p\right)
& \geq \min _{\pi \in \Delta_A^S}\left\{c^T q^\pi(p) \mid \pi \in \Pi_D\right\} 
= V^*\left(s_1\right)\\
& \geq \min _{\pi \in \Delta_A^S, p^{\prime} \in B_k^p}\left\{c^T q^\pi(p') \mid \pi \in \Pi_D^k\right\} \\
& =\min _{\pi \in \Delta_A^S, p^{\prime} \in B_k^p}\left\{c^T q^\pi(p') \mid {D} q^\pi\left(p^{\prime}\right) \leq \alpha\right\} 
=V_1^{\pi_k}\left(s_1 ; {c}, \widetilde{p}_k\right)
\end{aligned}
$$

なので、両方成立します。

## 拡張線形計画問題

OptCMDPは次の問題を解くことを思い出しましょう。

$$
\begin{aligned}
\min _{\widetilde{p} \in B_k^p, \pi \in \Pi^{\mathrm{MR}}} & \sum_{h, s, a} {c}_h(s, a) q_h^\pi(s, a ; \widetilde{p}) \\
\text { s.t. } & \sum_{h, s, a} {d}_{i, h}(s, a) q_h^\pi(s, a ; \widetilde{p}) \leq \alpha_i, \quad \forall i \in[H]
\end{aligned}
$$

一方で、これはダイレクトに解くことはできません（目的変数が２つあるせい？）。

そこで、

$$z_h^\pi\left(s, a, s^{\prime} ; p\right)=p_h\left(s^{\prime} \mid s, a\right) q_h^\pi(s, a ; p)$$

で定義される変数に対する拡張線形計画問題を解きます。
$$
\begin{array}{lll}
\min _z \sum_{h, s, a, s^{\prime}} z_h\left(s, a, s^{\prime}\right) c_h(s, a) & \\
\text { s.t. } \sum_{h, s, a, s^{\prime}} z_h\left(s, a, s^{\prime}\right) d_{i, h}(s, a) \leq \alpha_i & \forall i \in[I]\\
\sum_{a, s^{\prime}} z_h\left(s, a, s^{\prime}\right)=\sum_{s^{\prime}, a^{\prime}} z_{h-1}\left(s^{\prime}, a^{\prime}, s\right) & \forall h \in[H] \backslash\{1\} \\
\sum_{a, s^{\prime}} z_1\left(s, a, s^{\prime}\right)=\mu(s) & \forall s \in \mathcal{S}\\
z_h\left(s, a, s^{\prime}\right) \geq 0 & \forall\left(s, a, s^{\prime}, h\right) \in \mathcal{S} \times \mathcal{A} \times \mathcal{S} \times[H] \\
z_h\left(s, a, s^{\prime}\right)-\left(\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)+\beta_{h, k}^p\left(s, a, s^{\prime}\right)\right) \sum_y z_h(s, a, y) \leq 0 & \forall\left(s, a, s^{\prime}, h\right) \in \mathcal{S} \times \mathcal{A} \times \mathcal{S} \times[H] \\
-z_h\left(s, a, s^{\prime}\right)+\left(\bar{p}_h^{k-1}\left(s^{\prime} \mid s, a\right)-\beta_{h, k}^p\left(s, a, s^{\prime}\right)\right) \sum_y z_h(s, a, y) \leq 0 & \forall\left(s, a, s^{\prime}, h\right) \in \mathcal{S} \times \mathcal{A} \times \mathcal{S} \times[H]
\end{array}
$$

この解の$z$を次のようにすれば最適解が得られます。
$$\widetilde{p}_h^k\left(s^{\prime} \mid s, a\right)=\frac{z\left(s, a, s^{\prime}\right)}{\sum_y z(s, a, y)} \quad \text{and} \quad \pi_k(a \mid s)=\frac{\sum_{s^{\prime}} z\left(s, a, s^{\prime}\right)}{\sum_{b, s^{\prime}} z\left(s, b, s^{\prime}\right)}$$

このOptCMDPのリグレットを導出しましょう。

$\delta\in (0, 1)$とします。このとき、$K'\in [K]$について、確率$1-\delta$以上で、

$$
\begin{aligned}
& \operatorname{Reg}_{+}\left(K^{\prime} ; c\right) \leq \widetilde{\mathcal{O}}\left(\sqrt{S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right), \\
& \operatorname{Reg}_{+}\left(K^{\prime} ; d\right) \leq \widetilde{\mathcal{O}}\left(\sqrt{S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right) .
\end{aligned}
$$

が成り立ちます。証明しましょう。

まず、確率$1-\delta$以上でGood Eventが成立します（Good event参照）。
このとき、

$$
\begin{aligned}
\text { Regret }^{+}\left(K^{\prime} ; c\right) & =\sum_k\left[V_1^{\pi_k}\left(s_1 ; c, p\right)-V_1^*\left(s_1 ; c, p\right)\right]_{+} \leq \sum_k\left[V_1^{\pi_k}\left(s_1 ; c, p\right)-V_1^{\pi_k}\left(s_1 ; \widetilde{c}_k, \widetilde{p}_k\right)\right]_{+} \\
& =\sum_k V_1^{\pi_k}\left(s_1 ; c, p\right)-V_1^{\pi_k}\left(s_1 ; \widetilde{c}_k, \widetilde{p}_k\right) \\
& \leq \widetilde{\mathcal{O}}\left(\sqrt{S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right) .
\end{aligned}
$$

が成り立ちます。
ここで、最初の不等式と２つ目の等式は楽観的方策のおかげです。最後の不等式はAppendix Eを参照してください。（TODO: Appendix Eは何かと便利そうなのでまとめておこう。）

続いて制約違反のリグレットを導出します。
これも似たような変形でできます。
$$
\begin{aligned}
\sum_{k=1}^{K^{\prime}}\left[V_1^{\pi_k}\left(s_1 ; d_i, p\right)-\alpha_i\right]_{+} & =\sum_{k=1}^{K^{\prime}}[\underbrace{V_1^{\pi_k}\left(s_1 ; d_i, p\right)-V_1^{\pi_k}\left(s_1 ; \widetilde{d}_i^k, \tilde{p}^k\right)}_{\geq 0}+\underbrace{V_1^{\pi_k}\left(s_1 ; \tilde{d}_i^k, \tilde{p}^k\right)-\alpha_i}_{\leq 0}]_{+} \\
& \leq \sum_{k=1}^{K^{\prime}} V_1^{\pi_k}\left(s_1 ; d_i\right)-V_1^{\pi_k}\left(s_1 ; \tilde{d}_i^k, \widetilde{p}^k\right) \\
& \leq \widetilde{\mathcal{O}}\left(\sqrt{S \mathcal{N} H^4 K}+(\sqrt{\mathcal{N}}+H) H^2 S A\right) .
\end{aligned}
$$


PythonでOptCMDPを実装してみます。

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
from typing import NamedTuple, Optional
from jax.random import PRNGKey

key = PRNGKey(0)

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
H = 10  # ホライゾン

# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(H, S, A))
assert rew.shape == (H, S, A)


# コスト行列を適当に作ります
key, _ = jax.random.split(key)
cost = jax.random.uniform(key=key, shape=(H, S, A))
assert cost.shape == (H, S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(H, S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(H, S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 初期状態分布を適当に作ります
key, _ = jax.random.split(key)
init_dist = jax.random.uniform(key, shape=(S,))
init_dist = init_dist / jnp.sum(init_dist)
np.testing.assert_allclose(init_dist.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class CMDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    H: int  # ホライゾン
    rew: jnp.array  # 報酬行列
    cost: jnp.array  # 報酬行列
    const: float  # 制約の閾値
    P: jnp.array  # 遷移確率行列
    init_dist: jnp.array  # 初期分布
    optimal_Q_rew: Optional[jnp.ndarray] = None  # 最適Q値

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


const = 0.3 * H  # 制約は適当です。このときに実行可能である保証はとくにありません。
mdp = CMDP(S_set, A_set, H, rew, cost, const, P, init_dist)

print("状態数：", mdp.S)
print("行動数：", mdp.A)
print("ホライゾン：", mdp.H)
print("制約：", mdp.const)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


状態数： 10
行動数： 3
ホライゾン： 10
制約： 3.0


In [3]:
from functools import partial
import jax
import chex


@jax.jit
def compute_policy_Q(mdp: CMDP, policy: jnp.ndarray):
    """ベルマン期待作用素をホライゾン回走らせて価値関数を動的計画法で計算します。
    Args:
        mdp (CMDP)
        policy (np.ndarray): (HxSxA)の行列

    Returns:
        policy_Q_rew (jnp.ndarray): (HxSxA)の行列. 報酬関数についてのQ
        policy_Q_cost (jnp.ndarray): (HxSxA)の行列. コスト関数についてのQ
    """
    H, S, A = policy.shape

    def backup(i, args):
        policy_Q, g = args
        h = H - i - 1
        max_Q = (policy[h+1] * policy_Q[h+1]).sum(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        policy_Q = policy_Q.at[h].set(g[h] + next_v)
        return policy_Q, g
    
    policy_Q_rew = jnp.zeros((H+1, S, A))
    args = policy_Q_rew, mdp.rew
    policy_Q_rew, _ = jax.lax.fori_loop(0, mdp.H, backup, args)

    policy_Q_cost = jnp.zeros((H+1, S, A))
    args = policy_Q_cost, mdp.cost
    policy_Q_cost, _ = jax.lax.fori_loop(0, mdp.H, backup, args)
    return policy_Q_rew[:-1], policy_Q_cost[:-1]


uni_policy = jnp.ones((H, S, A)) / A
_ = compute_policy_Q(mdp, uni_policy)

まずは探索無しで、CMDPの最適方策を求めてみます

In [4]:
import pulp
from itertools import product
prob = pulp.LpProblem(name="CMDP", sense=pulp.LpMaximize)
hsa_indices = [(h, s, a) for h, s, a in product(range(H), range(S), range(A))]
sa_indices = [(s, a) for s, a in product(range(S), range(A))]
d = pulp.LpVariable.dicts("d", hsa_indices, lowBound=0, cat="Continuous")

# 目的関数
prob += pulp.lpSum([d[hsa] * mdp.rew[hsa[0], hsa[1], hsa[2]] for hsa in hsa_indices])

# 初期状態についての制約
for s in range(S):
    d_0sa = [d[(0, s, a)] for a in range(A)]
    prob += pulp.lpSum(d_0sa) == mdp.init_dist[s].item()

# 各ステップについての制約
for h in range(1, H):
    for ns in range(S):
        d_hns = pulp.lpSum([d[(h, ns, na)] for na in range(A)])
        d_phns = pulp.lpSum([d[(h-1, sa[0], sa[1])] * mdp.P[h-1, sa[0], sa[1], ns] for sa in sa_indices])
        prob += d_hns == d_phns

# CMDPとしての制約
const = pulp.lpSum([d[hsa] * mdp.cost[hsa[0], hsa[1], hsa[2]] for hsa in hsa_indices]) <= mdp.const
prob += const


sol = prob.solve()
d_arr = jnp.array([pulp.value(d[h, s, a]) for (h, s, a) in hsa_indices])
d_arr = d_arr.reshape(H, S, A)

np.testing.assert_allclose(d_arr.sum(axis=(1, 2)), 1.0, atol=1e-4)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/toshinori/.cache/pypoetry/virtualenvs/shumi-VTLwuKSy-py3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c6618811375d4308b2934efba067f2af-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/c6618811375d4308b2934efba067f2af-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 106 COLUMNS
At line 3707 RHS
At line 3809 BOUNDS
At line 3810 ENDATA
Problem MODEL has 101 rows, 300 columns and 3300 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 101 (0) rows, 300 (0) columns and 3300 (0) elements
0  Obj -0 Primal inf 1.4382028 (10) Dual inf 380.18679 (300)
0  Obj -0 Primal inf 1.4382028 (10) Dual inf 1.9122887e+12 (300)
31  Obj -0 Primal inf 1.4382028 (10) Dual inf 2.3956333e+12 (208)
62  Obj -0 Primal inf 1.4382028 (10) Dual inf 2.9382449e+12 (127)
93  Obj 1.1159012 Primal inf 1.

In [5]:
optimal_policy = d_arr / d_arr.sum(axis=-1, keepdims=True)
Q_rew, Q_cost = compute_policy_Q(mdp, optimal_policy)

total_cost = (Q_cost * optimal_policy)[0].sum(axis=-1) @ mdp.init_dist
assert total_cost <= mdp.const
print("最適方策の累積コスト和", total_cost)

total_rew = (Q_rew * optimal_policy)[0].sum(axis=-1) @ mdp.init_dist
print("最適方策の累積報酬和", total_rew)


最適方策の累積コスト和 3.0
最適方策の累積報酬和 6.65394


In [6]:
uniform_policy = jnp.ones((H, S, A)) / A
Q_rew, Q_cost = compute_policy_Q(mdp, uniform_policy)

total_cost = (Q_cost * uniform_policy)[0].sum(axis=-1) @ mdp.init_dist
print("一様方策の累積コスト和", total_cost)

total_rew = (Q_rew * uniform_policy)[0].sum(axis=-1) @ mdp.init_dist
print("一様方策の累積報酬和", total_rew)


一様方策の累積コスト和 4.9952
一様方策の累積報酬和 4.922522


最適方策は計算できました。OptCMDPを実装しましょう。